## PROJECT GOAL:
Outbrain is a web advertising platform that displays advertisement boxes of links to pages within websites. It displays links to the sites' pages in addition to sponsored content, generating revenue from the latter.
In our project we are predicting whether the particular ad_id will be clicked or not.

This python files predict ad using random forest classifier algorithm and measure Accuracy and AUC.

## Creating a Classification Model using Gradient Boosting Algorithm

In this file, we will implement a classification model using *Gradient Boosting that uses features of a Event,Document and Train dataframe to **predict the whether the ad is clicked or not** 

You should follow the steps below to build, train and test the model from the source data:

1. Build a schema of a source data for its Data Frame
2. Load the Source Data to the schema
3. Prepare the data with the features (input columns, output column as label)
4. Split the data using data.randomSplit(): Training and Testing
5. Transform the columns to a vector using VectorAssembler
6. set features and label from the vector
7. Build a ***Gradient Boosting Algorithm*** Model with the label and features
8. Train the model
9. Prepare the testing Data Frame with features and label from the vector; Rename label to trueLabel
10. Predict and test the testing Data Frame using the model trained at the step 8
11. Compare the predicted result and trueLabel


### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [3]:
# Import Spark SQL and Spark ML libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.mllib.tree import RandomForest
from pyspark.ml import Pipeline

from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier


#####The below command will start spark session when we run our file in oracle BDE. In Databricks keep this cell as False by default. But when you run file in Oracle BDE make it True.

In [5]:
#This 
IS_SPARK_SUBMIT_CLI = False
if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

### Load Source Data
The data for this project is provided as a CSV file containing details of advertisement. The data includes specific characteristics (or *features*) for each ad, as well as a *label* column indicating whether the ad was clicked or not.

You will load this data into a DataFrame and display it.

#####Reading all necessary csv file

In [8]:
if IS_SPARK_SUBMIT_CLI:
    event = spark.read.csv('eventd.csv', inferSchema=True, header=True)
else:
    event = spark.sql("SELECT * FROM eventd_csv")

In [9]:
if IS_SPARK_SUBMIT_CLI:
    train = spark.read.csv('traind.csv', inferSchema=True, header=True)
else:
    train = spark.sql("SELECT * FROM traind_csv")

In [10]:
if IS_SPARK_SUBMIT_CLI:
    promoted_content = spark.read.csv('promoted_content.csv', inferSchema=True, header=True)
else:
    promoted_content = spark.sql("SELECT * FROM promoted_content_csv")

Below command will join event and train file using disply_id as primary key. It will also drop display_id from event table and ad_id from train table as it create ambiguity error. Then it will assign result to new dataframe data1.

In [12]:
data1=event.join(train,event.display_id==train.display_id).drop(event.display_id).drop(train.ad_id)

Following cell will join two dataframe data1 (which created from joining of event and train table) with Promoted content dataframe using document_id as primary key. Then it will assign result to new dataframe data2.

In [14]:
data2=data1.join(promoted_content,data1.document_id==promoted_content.document_id).drop(data1.document_id)

Following table select disply_id,document_id,platform,ad_id,camaign_id,advertiser_id which are features which help us to train our dataset and clicked column which is label column. Selected column will assign to new dataframe called data.

In [16]:
data = data2.select("display_id", "document_id", "platform", "ad_id", "campaign_id","advertiser_id",col("clicked").alias("label"))

### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [18]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

### Prepare the Training Data
To train the classification model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **VectorAssembler** class to transform the feature columns into a vector, and then rename the **ArrDelay** column to **label**.

In [20]:
from pyspark.ml.classification import RandomForestClassifier

assembler = VectorAssembler(inputCols = ["display_id", "document_id", "platform", "ad_id","campaign_id","advertiser_id"],outputCol="normfeatures")
#assembler = VectorAssembler(inputCols = ["clicked"],outputCol="label")
minMax = MinMaxScaler(inputCol = assembler.getOutputCol(), outputCol="nfeatures")
featVect = VectorAssembler(inputCols=["nfeatures"], outputCol="features")
#rf = RandomForestClassifier(labelCol="label", featuresCol="features",impurity="gini",featureSubsetStrategy="auto",numTrees=10,maxDepth=32,maxBins=128,seed=1234)
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)
pipeline = Pipeline(stages=[assembler,minMax,featVect,gbt])

The pipeline itself is an estimator, and so it has a fit method that you can call to run the pipeline on a specified DataFrame. In this case, you will run the pipeline on the training data to train a model.

In [22]:
piplineModel = pipeline.fit(train)
print("Pipeline complete!")

Pipeline complete!

### Test the Recommender
The model produced by the pipeline is a transformed that will apply to all stages in the pipeline to a specified DataFrame and apply the trained model to generate predictions. In this case, you will transform the test DataFrame using the pipeline to generate label prediction

In [24]:
prediction = piplineModel.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

+---------------------------------------------------------------------------------------------------------+----------+---------+
features |prediction|trueLabel|
+---------------------------------------------------------------------------------------------------------+----------+---------+
[0.23366817877427618,0.49752889309438236,0.0,0.28616465912801087,0.5789429127747834,0.38717002554640934] |0.0 |1 |
[0.23366817877427618,0.49752889309438236,0.0,0.3181910736791847,0.5789429127747834,0.38717002554640934] |0.0 |1 |
[0.23366817877427618,0.49752889309438236,0.0,0.31821038256162165,0.5789429127747834,0.38717002554640934] |0.0 |1 |
[0.23366817877427618,0.49752889309438236,0.0,0.31821564862046814,0.5789429127747834,0.38717002554640934] |0.0 |1 |
[0.5977541407836479,0.7981802931965557,0.0,0.8082873723419568,0.020966786127493864,0.002554640931024695] |0.0 |0 |
[0.7016187812154383,0.7550586058765141,1.0,0.7865596135414948,0.34760278804639216,0.08174850979279023] |0.0 |0 |
[0.10106117287015641,0.27088882553485527,0.5,0.0778270836917179,0.14343764991393176,0.01107011070110701] |1.0 |0 |
[0.10106117287015641,0.27088882553485527,0.5,0.07783410510351316,0.05909078082230437,0.01107011070110701]|1.0 |0 |
[0.10106117287015641,0.27088882553485527,0.5,0.0778376158094108,0.05909078082230437,0.01107011070110701] |1.0 |0 |
[0.18811473807436094,0.27088882553485527,0.5,0.07783234975056434,0.05909078082230437,0.01107011070110701]|1.0 |0 |
[0.18811473807436094,0.27088882553485527,0.5,0.07783410510351316,0.05909078082230437,0.01107011070110701]|1.0 |0 |
[0.18811473807436094,0.27088882553485527,0.5,0.0778376158094108,0.05909078082230437,0.01107011070110701] |1.0 |0 |
[0.524338853152041,0.27088882553485527,0.0,0.0778270836917179,0.14343764991393176,0.01107011070110701] |0.0 |1 |
[0.524338853152041,0.27088882553485527,0.0,0.07783234975056434,0.05909078082230437,0.01107011070110701] |0.0 |1 |
[0.5587251757521771,0.27088882553485527,1.0,0.07782883904466671,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.5587251757521771,0.27088882553485527,1.0,0.07873811187215414,0.001890679233569433,0.01107011070110701]|0.0 |0 |
[0.6057386124259493,0.27088882553485527,1.0,0.0778270836917179,0.14343764991393176,0.01107011070110701] |0.0 |1 |
[0.6057386124259493,0.27088882553485527,1.0,0.07783234975056434,0.05909078082230437,0.01107011070110701] |0.0 |1 |
[0.6057386124259493,0.27088882553485527,1.0,0.07783410510351316,0.05909078082230437,0.01107011070110701] |0.0 |1 |
[0.6679213408183441,0.8864441262362276,0.5,0.887476609921957,0.02342184722183029,0.04598353675844451] |0.0 |0 |
[0.6679213408183441,0.8864441262362276,0.5,0.8877715092173584,0.0027372520247199257,0.04598353675844451] |0.0 |0 |
[0.9036495805567077,0.27088882553485527,1.0,0.0778270836917179,0.14343764991393176,0.01107011070110701] |0.0 |0 |
[0.9036495805567077,0.27088882553485527,1.0,0.07782883904466671,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9036495805567077,0.27088882553485527,1.0,0.07783234975056434,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9036495805567077,0.27088882553485527,1.0,0.07783410510351316,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9036495805567077,0.27088882553485527,1.0,0.07783586045646199,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9450505050744769,0.27088882553485527,1.0,0.0778270836917179,0.14343764991393176,0.01107011070110701] |0.0 |0 |
[0.9450505050744769,0.27088882553485527,1.0,0.07783059439761553,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9450505050744769,0.27088882553485527,1.0,0.07873811187215414,0.001890679233569433,0.01107011070110701]|0.0 |0 |
[0.5435174974358998,0.4798969781923512,0.0,0.2682109091675063,0.13502836018850353,0.4175418677263696] |1.0 |0 |
[0.6938315229434677,0.897265802087831,1.0,0.8961831605480914,0.7301408132742614,0.027817201248935566] |0.0 |0 |
[0.17908065700530784,0.6921554803855552,0.0,0.6747524074665693,0.0027372520247199257,0.04598353675844451]|0.0 |0 |
[0.244645084459711,0.6921554803855552,1.0,0.6727

Following cell measure accuracy of Algorithm using MultiClassificationEvaluater and evaluate using predicted data.

In [26]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluation = MulticlassClassificationEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluation.evaluate(prediction)
print(accuracy)

0.8978065802592223

Following Code will give total Error in project

In [28]:
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.102193

#### Gradient Boosting Evaluator
Calculate AUC using Gradient Boosting Evaluator.

In [30]:
rf_evaluator =  MulticlassClassificationEvaluator(labelCol="trueLabel", predictionCol="prediction")
rf_auc = rf_evaluator.evaluate(prediction)
print("AUC for Gradient Boost is= ", rf_auc)

AUC for Gradient Boost is= 0.8760900114611925